In [2]:
import requests
import folium
import json
import pystac
import rioxarray
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl
import fsspec
import geopandas as gpd
import branca
from branca.colormap import LinearColormap

In [3]:
tile_names = ['10TGM',
 '10TGL',
 '10TGP',
 '10TGN',
 '10TGQ',
 '10TFP',
 '10TFT',
 '10TFN',
 '10TGT',
 '10TFS',
 '10TFM',
 '10TGK',
 '10TFL',
 '10TES',
 '10TEQ',
 '10TDT',
 '10TEP',
 '10TEN',
 '10TFK',
 '10TEL',
 '10TDL',
 '10TGR',
 '10TFQ',
 '10TET',
 '10TDM',
 '10TGS',
 '10TEM',
 '10TFR',
 '10TER']

In [4]:
cm = branca.colormap.linear.viridis.scale(0,365)
#cm.colors.reverse()
#cm = branca.colormap.LinearColormap(colors=cm.colors, caption='sqrt(gamma0)')
#cm
#from rio_tiler.colormap import cmap
#cmap.list()
#

In [9]:
m = folium.Map(
    location=(40.76, -111.89),
    tiles="Stamen Terrain", #"cartodbpositron"
    zoom_start=7,
)

interannual_medians = folium.FeatureGroup(name='2015-2022 Median Snowmelt Runoff Onset Date')
interannual_std = folium.FeatureGroup(name='2015-2022 Standard Deviation of Snowmelt Runoff Onset Date',show=False)
interannual_polyfit = folium.FeatureGroup(name='2015-2022 Snowmelt Runoff Onset Date Trend',show=False)
interannual_corr = folium.FeatureGroup(name='2015-2022 Snowmelt Runoff Onset Date Trend Strength',show=False)
year_2015 = folium.FeatureGroup(name='2015 Snowmelt Runoff Onset Date',show=False)
year_2016 = folium.FeatureGroup(name='2016 Snowmelt Runoff Onset Date',show=False)
year_2017 = folium.FeatureGroup(name='2017 Snowmelt Runoff Onset Date',show=False)
year_2018 = folium.FeatureGroup(name='2018 Snowmelt Runoff Onset Date',show=False)
year_2019 = folium.FeatureGroup(name='2019 Snowmelt Runoff Onset Date',show=False)
year_2020 = folium.FeatureGroup(name='2020 Snowmelt Runoff Onset Date',show=False)
year_2021 = folium.FeatureGroup(name='2021 Snowmelt Runoff Onset Date',show=False)
year_2022 = folium.FeatureGroup(name='2022 Snowmelt Runoff Onset Date',show=False)


tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_medians.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2015-2022 Median Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_std_40m.tif"
    expression = "rescale=0,90&colormap_name=reds&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_std.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, opacity=1,name='2015-2022 STD Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))

    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_polyfit_40m.tif"
    expression = f"rescale=-10,10&colormap_name=rdbu_r&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_polyfit.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, opacity=1,name='2015-2022 Snowmelt Runoff Onset Trend', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
 
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_corr_strength_40m.tif"
    expression = "rescale=-1,1&colormap_name=rdbu_r&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_corr.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2015-2022 Snowmelt Runoff Onset Trend Strength', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))

    

    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2015_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2015.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2015 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2016_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2016.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2016 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))

for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2017_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2017.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2017 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2018_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2018.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2018 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2019_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2019.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2019 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2020_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2020.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2020 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2021_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2021.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2021 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_2022_median_40m.tif"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    year_2022.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2022 Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
    
    
interannual_medians.add_to(m)
interannual_std.add_to(m)
interannual_polyfit.add_to(m)
interannual_corr.add_to(m)
year_2015.add_to(m)
year_2016.add_to(m)
year_2017.add_to(m)
year_2018.add_to(m)
year_2019.add_to(m)
year_2020.add_to(m)
year_2021.add_to(m)
year_2022.add_to(m)



folium.LayerControl().add_to(m)
cm.add_to(m)

m

In [5]:
m.save("index.html")